In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## Qwen2-VL-finetune

### Download model

In [2]:
model_id_3b = 'Qwen/Qwen2.5-VL-3B-Instruct-AWQ'
model_id_7b = 'Qwen/Qwen2.5-VL-7B-Instruct-AWQ'
save_dir_3b = './vl3b/'
save_dir_7b = './vl7b/'

In [3]:
# !pip install git+https://github.com/huggingface/transformers
# !pip install qwen-vl-utils
# !pip install openai
# !pip install flash-attn --no-build-isolation
# !pip install swanlab  # monitor
# !swanlab login

In [2]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id='Qwen/Qwen2.5-VL-3B-Instruct-AWQ', local_dir='./vl3b/', local_dir_use_symlinks=False)
snapshot_download(repo_id='Qwen/Qwen2.5-VL-7B-Instruct-AWQ', local_dir='./vl7b/', local_dir_use_symlinks=False)

/root/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

'/root/autodl-tmp/HOCR/finetune/vl7b'

In [3]:
import transformers
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, AutoTokenizer

model_7b = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    save_dir_7b, torch_dtype=torch.float16, 
    attn_implementation="flash_attention_2", device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(save_dir_7b, trust_remote=True)
processor = AutoProcessor.from_pretrained(save_dir_7b)

model_7b.enable_input_require_grads()   # 开启梯度检查点时，要执行该方法

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
print(model_7b)

Qwen2_5_VLForConditionalGeneration(
  (visual): Qwen2_5_VisionTransformerPretrainedModel(
    (patch_embed): Qwen2_5_VisionPatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2_5_VLVisionBlock(
        (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
        (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
        (attn): Qwen2_5_VLVisionFlashAttention2(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): Qwen2_5_VLMLP(
          (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
          (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
          (act_fn): SiLU()
        )
      )
    )
    (merger): Qwen2_5_VLPatchMerg

In [2]:
from qwen_vl_utils import process_vision_info

def preprocess_func(example):
    MAX_LENGTH = 8192
    input_ids, attention_mask, labels = [], [], []
    url = example["message"][0]["conversation"][0]['url']
    caption = example["message"][0]["conversation"][1]['caption']
    
    messages = [
        {
            "role": "system", 
            "content": "You are a helpful assistant in recognize math equations in either handwritten or printed text."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Recognize the equation in the image, write its LaTeX code bettwen $$\t and \t$$"
                },
                {
                    "type": "image",
                    "image": url,
                    "resized_height": 280,
                    "resized_width": 280,
                },
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": caption
                }
            ]
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    img_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=img_inputs,
        padding=True,
        return_tensors='pt'
    )
    inputs = {key: value.tolist() for key, value in inputs.items()}
    instruction = inputs
    response = tokenizer(f'{caption}', add_special_tokens=False)
    input_ids = (
        instruction["input_ids"][0] + response['input_ids'] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction['attention_mask'][0] + response['attention_mask'] + [1]
    labels = (
        [-100] * len(instruction['input_ids'][0])
        + response['input_ids']
        + [tokenizer.pad_token_id]
    )

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    labels = torch.tensor(labels)

    inputs['pixel_values'] = torch.tensor(inputs['pixel_values'])
    # 由 (1, h, w) 变换为 (h, w)
    inputs['image_grid_thw'] = torch.tensor(inputs['image_grid_thw']).squeeze(0)  
    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask, 
        "labels": labels,
        "pixel_values": inputs['pixel_values'], 
        "image_grid_thw": inputs['image_grid_thw']
    }

In [5]:
""" 数据集准备 """
import json
import random
from datasets import Dataset

dataset = './ft_data.json'
with open(dataset, 'r') as f:
    data = json.load(f)

# 设置 seed
random.seed(5525)
# shuffle 
random.shuffle(data)

train_set, test_set = data[:1080], data[1080:]

with open("ft_data_train.json", "w") as f:
    json.dump(train_set, f)

with open("ft_data_test.json", "w") as f:
    json.dump(test_set, f)

In [6]:
train_data = Dataset.from_json('ft_data_train.json')
train_data = train_data.map(preprocess_func)
print(train_data)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Dataset({
    features: ['message', 'input_ids', 'attention_mask', 'labels', 'pixel_values', 'image_grid_thw'],
    num_rows: 1080
})


In [7]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

# 配置LoRA
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=64,  # Lora 秩
    lora_alpha=16,
    lora_dropout=0.05,  # Dropout 比例
    bias="none",
)

# 获取LoRA模型
peft_model_7b = get_peft_model(model_7b, config)

In [8]:
from transformers import (
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    Qwen2VLForConditionalGeneration,
    AutoProcessor,
)

# 配置训练参数
args = TrainingArguments(
    output_dir="./output/Qwen2.5-VL-7B",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    logging_first_step=5,
    num_train_epochs=2,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

In [9]:
import swanlab
from swanlab.integration.transformers import SwanLabCallback

# 设置SwanLab回调
swanlab_callback = SwanLabCallback(
    project="Qwen2.5-VL-7b-finetune",
    experiment_name="qwen2.5-vl-crohme2019",
    config={
        "model": "https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct-AWQ",
        "dataset": "https://disk.pku.edu.cn/anyshare/en-us/link/AAF10CCC4D539543F68847A9010C607139?_tb=none&expires_at=1970-01-01T08%3A00%3A00%2B08%3A00&item_type=&password_required=false&title=HMER%20Dataset&type=anonymous",
        "github": "https://github.com/Wooonster/HOCR",
        "prompt": "https://github.com/Wooonster/HOCR",
        "train_data_number": len(train_data),
        "lora_rank": 64,
        "lora_alpha": 16,
        "lora_dropout": 0.1,
    },
)

In [10]:
# 配置Trainer
trainer = Trainer(
    model=peft_model_7b,
    args=args,
    train_dataset=train_data,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

# 开启模型训练
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


swanlab: Using SwanLab to track your experiments. Please refer to https://docs.swanlab.cn for more information.
swanlab: (1) Create a SwanLab account.
swanlab: (2) Use an existing SwanLab account.
swanlab: (3) Don't visualize my results.


swanlab: Enter your choice:  2


swanlab: You chose 'Use an existing swanlab account'
swanlab: Logging into https://swanlab.cn
swanlab: You can find your API key at: https://swanlab.cn/space/~/settings
swanlab: Paste an API key from your profile and hit enter, or press 'CTRL + C' to quit: 


 ········


swanlab: Tracking run with swanlab version 0.4.8                                   
swanlab: Run data will be saved locally in /root/autodl-tmp/HOCR/finetune/swanlog/run-20250302_130228-a3b1799d
swanlab: 👋 Hi Wonster, welcome to swanlab!
swanlab: Syncing run qwen2.5-vl-crohme2019 to the cloud
swanlab: 🌟 Run `swanlab watch /root/autodl-tmp/HOCR/finetune/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@Wonster/Qwen2.5-VL-7b-finetune
swanlab: 🚀 View run at https://swanlab.cn/@Wonster/Qwen2.5-VL-7b-finetune/runs/1jxoubmr2r64w1ax2usme


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/ch

Step,Training Loss
1,1.482700
10,0.399600
20,0.003000
30,0.000400
40,0.000200
50,0.000100
60,0.000100
70,0.000100
80,0.000100
90,0.000100


/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=134, training_loss=0.03823906746966618, metrics={'train_runtime': 1161.3783, 'train_samples_per_second': 1.86, 'train_steps_per_second': 0.115, 'total_flos': 4362943305351168.0, 'train_loss': 0.03823906746966618, 'epoch': 1.9777777777777779})

In [4]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

# 配置测试参数
val_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=True,  # 训练模式
    r=64,  # Lora 秩
    lora_alpha=16,
    lora_dropout=0.05,  # Dropout 比例
    bias="none",
)

# 获取测试模型
val_peft_model = PeftModel.from_pretrained(model_7b, model_id="./output/Qwen2-VL-7B/checkpoint-134", config=val_config)

NameError: name 'model_7b' is not defined

In [13]:
def predict(messages, model):
    # 准备推理
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # 生成输出
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    
    return output_text[0]

In [ ]:
# 读取测试数据
with open("ft_data_test.json", "r") as f:
    test_dataset = json.load(f)

test_image_list = []
predict_results = {}
for item in test_dataset:
    url = item["message"][0]["conversation"][0]['url']
    # caption = item["message"][0]["conversation"][1]['caption']
    
    messages = [
        {
            "role": "system", 
            "content": "You are a helpful assistant in recognize math equations in either handwritten or printed text."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Recognize the equation in the image, write its LaTeX code between $$\t and \t$$"
                },
                {
                    "type": "image",
                    "image": url,
                    "resized_height": 280,
                    "resized_width": 280,
                },
            ]
        }
    ]
    
    response = predict(messages, val_peft_model)
    messages.append({"role": "assistant", "content": f"{response}"})
    print(messages[-1])

    predict_results[url] = response

    test_image_list.append(swanlab.Image(url, caption=response))

swanlab.log({"Prediction": test_image_list})

# 在Jupyter Notebook中运行时要停止SwanLab记录 需要调用swanlab.finish()
swanlab.finish()

/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv3d(


{'role': 'assistant', 'content': '$$x = x_a - x_b$$'}
{'role': 'assistant', 'content': '$$a=1...7$$'}
{'role': 'assistant', 'content': '$$x+u$$'}
{'role': 'assistant', 'content': '$$\\sqrt{1 + z^2}$$'}
{'role': 'assistant', 'content': '$$C_{t}=K$$'}
{'role': 'assistant', 'content': '$$\\text{and } \\text{one goes down from } m$$'}
{'role': 'assistant', 'content': '$$\\frac{1}{\\epsilon}\\int_{-\\infty}^{\\infty}dz$$'}
{'role': 'assistant', 'content': '$$a b - a ^ { - 2 } b ^ { - 2 }$$'}
{'role': 'assistant', 'content': '$$b_{m}=\\lim _{a\\to 0}{\\frac {b_{m}-a}{a}}$$'}
{'role': 'assistant', 'content': '$$\\Delta ^ { \\prime } ( m ) = 8 x - \\frac { 1 } { 6 } ( m + 1 ) ( m + 2 ) ( m + 3 )$$'}
{'role': 'assistant', 'content': '$$\\sqrt{\\frac{R}{n}}$$'}
{'role': 'assistant', 'content': '$$\\frac{-4}{\\sqrt{360}}$$'}
{'role': 'assistant', 'content': '$$\\sqrt{-8}$$'}
{'role': 'assistant', 'content': '$$\\lim_{R \\to 0} k^2 G(R) = \\infty$$'}
{'role': 'assistant', 'content': '$$\\int d^dx 

In [ ]:
from test_funcs import compute_bleu, compute_exprate

compute_bleu(predict_results)
compute_exprate(predict_results)